In [3]:
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import norm, truncnorm
from scipy.special import erf
from scipy.odr import ODR, Model, RealData

from dotenv import load_dotenv
load_dotenv(override=True)

ROOT_PATH = os.environ.get('ROOT_PATH')
if not ROOT_PATH in sys.path: sys.path.append(ROOT_PATH)

from main_code.utils.constants import SURVEY_LIST, SURVEY_VELDISP_LIMIT, LIGHTSPEED, SOLAR_MAGNITUDE, MAG_HIGH, MAG_LOW
from main_code.utils.CosmoFunc import rz_table
from main_code.utils.CosmoFunc import FP_func as FP_func_ori
from experiments.experiment_020_fix_abc.rmean_fixed.rmean_dependent import FP_func as FP_func_abc_fixed

pvhub_dir = os.environ.get('PVHUB_DIR_PATH')
if not pvhub_dir in sys.path: sys.path.append(pvhub_dir)
from pvhub import TwoMPP_SDSS_6dF

# 1. Calculate AIC and BIC for the combined FP fit

$$AIC = 2k -2\ln \mathcal{\hat{L}}$$

$$BIC = k\ln n -2\ln \mathcal{\hat{L}}$$

In [4]:
# Load best-fit combined FP
fp_params = pd.read_csv("../../artifacts/fp_fit/smin_setting_1/fp_fit_method_0/fp_fits.csv", index_col=0).loc["ALL_COMBINED"].to_numpy()

# Load combined dataset
df = pd.read_csv("../../data/foundation/fp_sample_final/smin_setting_1/fp_fit_method_0/all_combined.csv")

# Load PV model
pv_model = TwoMPP_SDSS_6dF(verbose=True) # type: ignore

# Calculate predicted PVs using observed group redshift in CMB frame, and calculate cosmological redshift
df['v_pec'] = pv_model.calculate_pv(df['ra'].to_numpy(), df['dec'].to_numpy(), df['z_dist_est'].to_numpy())
df['z_pec'] = df['v_pec'] / LIGHTSPEED
df['z_cosmo'] = ((1 + df['z_dist_est']) / (1 + df['z_pec'])) - 1

# Calculate predicted true distance and FN integral limits
red_spline, lumred_spline, dist_spline, lumdist_spline, ez_spline = rz_table()
d_H = sp.interpolate.splev(df['z_cosmo'].to_numpy(), dist_spline, der=0)
df['lmin'] = (SOLAR_MAGNITUDE["j"] + 5.0 * np.log10(1.0 + df["zhelio"].to_numpy()) + df["kcor_j"].to_numpy() + df["extinction_j"].to_numpy() + 10.0 - 2.5 * np.log10(2.0 * np.pi) + 5.0 * np.log10(d_H) - MAG_HIGH) / 5.0
df['lmax'] = (SOLAR_MAGNITUDE["j"] + 5.0 * np.log10(1.0 + df["zhelio"].to_numpy()) + df["kcor_j"].to_numpy() + df["extinction_j"].to_numpy() + 10.0 - 2.5 * np.log10(2.0 * np.pi) + 5.0 * np.log10(d_H) - MAG_LOW) / 5.0

# Load required data
z_cmb = df["z_cmb"].to_numpy()
logdists = 0.0
r = df["r"].to_numpy()
s = df["s"].to_numpy()
i = df["i"].to_numpy()
er = df["er"].to_numpy()
es = df["es"].to_numpy()
ei = df["ei"].to_numpy()
Sn = 1.0
smin = SURVEY_VELDISP_LIMIT[1]["6dFGS"]
lmin = df["lmin"].to_numpy()
lmax = df["lmax"].to_numpy()
C_m = 1.0


# Calculate log-likelihood at best-fit parameters
log_likelihood = FP_func_ori(
    params=fp_params,
    logdists=logdists,
    z_obs=z_cmb,
    r=r,
    s=s,
    i=i,
    err_r=er,
    err_s=es,
    err_i=ei,
    Sn=Sn,
    smin=smin,
    lmin=lmin,
    lmax=lmax,
    C_m=C_m,
    sumgals=True,
    chi_squared_only=False,
    use_full_fn=True
)

# Calculate AIC and BIC for the combined FP fit
k = 8
n = len(df)

AIC_combined = -2 * log_likelihood + 2 * k
print("AIC for the combined fit: ", AIC_combined)

BIC_combined = k * np.log(n) - 2 * log_likelihood
print("BIC for the combined fit: ", BIC_combined)

Loaded model 2M++_SDSS_6dF
AIC for the combined fit:  127823.55048830701
BIC for the combined fit:  127883.19933711762


# 2. Calculate AIC and BIC for the fixed slope FP fit

In [8]:
# Load PV model
pv_model = TwoMPP_SDSS_6dF(verbose=True) # type: ignore

# Load best-fit combined FP
fp_combined = pd.read_csv("../../artifacts/fp_fit/smin_setting_1/fp_fit_method_0/fp_fits.csv", index_col=0).loc["ALL_COMBINED"]
fp_combined["c"] = fp_combined["rmean"] - fp_combined["a"] * fp_combined["smean"] - fp_combined["b"] * fp_combined["imean"]
a, b, c = fp_combined[["a", "b", "c"]]

log_likelihood_sum = 0
n_sum = 0
for survey in SURVEY_LIST:
    # Load slope-fixed FP fits
    fp_params = pd.read_csv(f"../experiment_020_fix_abc/rmean_fixed/fp_fit.csv", index_col=0).loc[survey].to_numpy()

    # Load data
    df = pd.read_csv(f"../experiment_020_fix_abc/rmean_fixed/{survey.lower()}.csv")

    # Calculate predicted PVs using observed group redshift in CMB frame, and calculate cosmological redshift
    df['v_pec'] = pv_model.calculate_pv(df['ra'].to_numpy(), df['dec'].to_numpy(), df['z_dist_est'].to_numpy())
    df['z_pec'] = df['v_pec'] / LIGHTSPEED
    df['z_cosmo'] = ((1 + df['z_dist_est']) / (1 + df['z_pec'])) - 1

    # Calculate predicted true distance and FN integral limits
    red_spline, lumred_spline, dist_spline, lumdist_spline, ez_spline = rz_table()
    d_H = sp.interpolate.splev(df['z_cosmo'].to_numpy(), dist_spline, der=0)
    df['lmin'] = (SOLAR_MAGNITUDE["j"] + 5.0 * np.log10(1.0 + df["zhelio"].to_numpy()) + df["kcor_j"].to_numpy() + df["extinction_j"].to_numpy() + 10.0 - 2.5 * np.log10(2.0 * np.pi) + 5.0 * np.log10(d_H) - MAG_HIGH) / 5.0
    df['lmax'] = (SOLAR_MAGNITUDE["j"] + 5.0 * np.log10(1.0 + df["zhelio"].to_numpy()) + df["kcor_j"].to_numpy() + df["extinction_j"].to_numpy() + 10.0 - 2.5 * np.log10(2.0 * np.pi) + 5.0 * np.log10(d_H) - MAG_LOW) / 5.0

    # Load required data
    z_cmb = df["z_cmb"].to_numpy()
    logdists = 0.0
    r = df["r"].to_numpy()
    s = df["s"].to_numpy()
    i = df["i"].to_numpy()
    er = df["er"].to_numpy()
    es = df["es"].to_numpy()
    ei = df["ei"].to_numpy()
    Sn = 1.0
    smin = SURVEY_VELDISP_LIMIT[1]["6dFGS"]
    lmin = df["lmin"].to_numpy()
    lmax = df["lmax"].to_numpy()
    C_m = 1.0


    # Calculate log-likelihood at best-fit parameters
    log_likelihood = FP_func_abc_fixed(
        params=fp_params,
        logdists=logdists,
        z_obs=z_cmb,
        r=r,
        s=s,
        i=i,
        err_r=er,
        err_s=es,
        err_i=ei,
        Sn=Sn,
        smin=smin,
        lmin=lmin,
        lmax=lmax,
        C_m=C_m,
        sumgals=True,
        chi_squared_only=False,
        use_full_fn=True,
        a=a,
        b=b,
        c=c
    )
    log_likelihood_sum += log_likelihood
    n_sum += len(df)

# Calculate AIC and BIC for the combined FP fit
k = 3 + 3 * (5)

AIC_fixed_slope = -2 * log_likelihood_sum + 2 * k
print("AIC for the fixed slope fit: ", AIC_fixed_slope)

BIC_fixed_slope = k * np.log(n) - 2 * log_likelihood_sum
print("BIC for the fixed_slope fit: ", BIC_fixed_slope)

Loaded model 2M++_SDSS_6dF
AIC for the fixed slope fit:  128885.99744226335
BIC for the fixed_slope fit:  129020.20735208724
